In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("Churn_Modelling.csv")
original_df=df.copy()

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [6]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le=LabelEncoder()
df["Gender"]=le.fit_transform(df["Gender"])

In [9]:
df["Gender"]

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [10]:
df["Geography"].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [11]:
from sklearn.preprocessing import OneHotEncoder
oh=OneHotEncoder()

geo_encoded=oh.fit_transform(df[["Geography"]]).toarray()
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [12]:
oh.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
geo_encoded_df=pd.DataFrame(geo_encoded, columns=oh.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
df=pd.concat([df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [15]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [16]:
import pickle
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(le, file)

with open('one_hot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(oh, file)

In [17]:
X=df.drop('Exited', axis=1)
y=df['Exited']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [19]:
y_train

9254    0
1561    0
1670    1
6087    1
6669    1
       ..
5734    0
5191    0
5390    1
860     1
7270    0
Name: Exited, Length: 8000, dtype: int64

In [20]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [21]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [22]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [23]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN

In [24]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [25]:
import sys
sys.version

'3.10.19 (main, Oct 21 2025, 16:37:10) [Clang 20.1.8 ]'

In [26]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [27]:
(X_train.shape[1], )

(12,)

In [28]:
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1], )),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
opt=tensorflow.keras.optimizers.legacy.Adam(learning_rate=0.01)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
import datetime

In [31]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
logs="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=logs, histogram_freq=1)

In [32]:
early_stopping=EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

In [33]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping]
)

Epoch 1/100
250/250 [==============================] - 0s 856us/step - loss: 0.3893 - accuracy: 0.8393 - val_loss: 0.3474 - val_accuracy: 0.8565
Epoch 2/100
250/250 [==============================] - 0s 486us/step - loss: 0.3552 - accuracy: 0.8550 - val_loss: 0.3703 - val_accuracy: 0.8485
Epoch 3/100
250/250 [==============================] - 0s 487us/step - loss: 0.3462 - accuracy: 0.8605 - val_loss: 0.3503 - val_accuracy: 0.8560
Epoch 4/100
250/250 [==============================] - 0s 478us/step - loss: 0.3424 - accuracy: 0.8587 - val_loss: 0.3379 - val_accuracy: 0.8605
Epoch 5/100
250/250 [==============================] - 0s 526us/step - loss: 0.3391 - accuracy: 0.8630 - val_loss: 0.3422 - val_accuracy: 0.8565
Epoch 6/100
250/250 [==============================] - 0s 521us/step - loss: 0.3383 - accuracy: 0.8619 - val_loss: 0.3398 - val_accuracy: 0.8545
Epoch 7/100
250/250 [==============================] - 0s 498us/step - loss: 0.3338 - accuracy: 0.8658 - val_loss: 0.3413 - val_ac

In [34]:
model.save('model_1')

INFO:tensorflow:Assets written to: model_1/assets


INFO:tensorflow:Assets written to: model_1/assets


In [35]:
%load_ext tensorboard

In [36]:
%tensorboard --logdir logs/fit/20260114-101023